# City cube processing loop - get CLC Water classes in buffered city
Get land cover data from CLC 2018 in and around city, by creating a buffer around city geometry.
This notebook focuses on Water classes (CLC4**).

In [77]:
%matplotlib inline

import numpy as np
import fiona
from shapely.geometry import MultiLineString, MultiPolygon, Polygon, box, shape
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path
import rasterio as rio
import IPython.display
import time  
import datetime
from contextlib import redirect_stdout

# load utils functions
from src import utils

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    BBoxSplitter,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    SentinelHubDownloadClient,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
    Geometry
)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")

In [78]:
# read in the city bounding box
# dbfile = r'./../../../s3/data/d000_lookuptables/city_pts_urban_audit2021.sqlite'
csv_file = "./../../../s3/data/d000_lookuptables/URAU_city_bboxes_3035.csv"
df = gpd.read_file(csv_file)
df.drop(df[df.URAU_NAME == "Cayenne"].index,inplace=True)
df[df.URAU_NAME == "Thun"].miny

0    2624628.388800001
Name: miny, dtype: object

## Collections

In [79]:
collection_id = "1a624fad-9c1d-48a3-b2e4-82d71e0a58c8"
collection_name="env_zones"
# define collection
data_collection = DataCollection.define_byoc(collection_id, name=collection_name)
data_collection

<DataCollection.env_zones: DataCollectionDefinition(
  api_id: byoc-1a624fad-9c1d-48a3-b2e4-82d71e0a58c8
  catalog_id: byoc-1a624fad-9c1d-48a3-b2e4-82d71e0a58c8
  wfs_id: byoc-1a624fad-9c1d-48a3-b2e4-82d71e0a58c8
  collection_type: BYOC
  collection_id: 1a624fad-9c1d-48a3-b2e4-82d71e0a58c8
  is_timeless: False
  has_cloud_coverage: False
)>

## Prepare request
Define evalscript and an helper function to get the SentinelHub requests

In [80]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B01"],
    output: { 
        bands: 1,
        sampleType: "UINT16" // raster format will be UINT16
        }
    
  };
}

function evaluatePixel(sample) {
  return [sample.B01];
}
"""
input_data = [
        SentinelHubRequest.input_data(
            data_collection=DataCollection.env_zones,
            # time_interval=("2017-01-01", "2019-01-01") # select only CLC 2018
        ),
    ]
def sentinelhub_request(evalscript, input_data, bbox, bbox_size, config):
    request = SentinelHubRequest(
        evalscript=evalscript,
        input_data=input_data,
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=bbox_size,
        # geometry = geometry,
        config=config,
        )
    return request
# data = request.get_data()[0]
# images = request.get_data()[0]
# print(f"Image type: {images.dtype}")

# # plot function
# # factor 1/255 to scale between 0-1
# # factor 3.5 to increase brightness
# utils.plot_image(images)

In [90]:
from scipy import stats as st
# create temporary df
df_all = pd.DataFrame(columns=['urau_name', 'env_zone'])
bbox_coords = list(df[["minx", "miny", "maxx", "maxy"]].itertuples(index=False, name=None))
bboxs = [BBox(bbox=bbox_c, crs=CRS('3035').pyproj_crs()) for bbox_c in bbox_coords]
sh_requests = [(sentinelhub_request(evalscript,input_data, bbox, bbox_to_dimensions(bbox, resolution=1000), config),code) for bbox,code in list(zip(bboxs,df["URAU_CODE"]))]
for request,code in sh_requests:
    data = request.get_data()[0]
    val = st.mode(data.ravel(), keepdims=False).mode
    list_row = [code,val]
    df_all.loc[len(df_all)] = list_row
    print(code, val)
    # break
# optionally save results to temporary csv. useful when loop breaks because of SH request errors
df_all.to_csv("env_zones.csv", mode="a")

CH011C 6
CH012C 6
BE003C 7
BE004C 7
BG018C 8
FR027C 7
FR028C 13
FR029C 7
FR030C 0
FR044C 7
FR045C 7
FR046C 0
FR047C 9
BE005C 7
BG015C 8
BG014C 8
CH001C 6
HU006C 8
HU007C 8
HU008C 8
CY001C 13
CH002C 6
FI004C 2
FR031C 7
IT008C 12
FR048C 12
BE006C 7
BE007C 7
BE008C 7
AT001C 8
AT002C 6
FR032C 7
FR033C 12
IT006C 11
IT007C 12
BG016C 6
BE009C 7
CH003C 7
CH004C 7
CH005C 6
FR054C 9
FR055C 9
FR056C 7
FR057C 12
BE010C 7
BE011C 7
BE012C 7
BE013C 7
AT003C 6
AT004C 6
AT005C 5
FR049C 11
FR050C 5
FR051C 7
FR052C 7
BE014C 7
BE015C 6
CH006C 6
CH007C 6
CH008C 5
HR007C 12
HU001C 11
BG001C 6
HR004C 8
HR005C 12
HU013C 8
HU014C 8
HU015C 8
HU016C 8
BG002C 8
BG003C 8
BG004C 8
IE005C 7
FR017C 7
IT003C 12
IT004C 11
HU009C 8
HU010C 8
HU011C 8
HU012C 8
BG005C 8
BG006C 8
FR018C 7
FR019C 7
FR020C 7
FR021C 7
FI005C 2
FR001C 7
FR002C 7
FR053C 7
BG007C 8
HR006C 12
FR034C 7
FR035C 7
FR036C 7
FR037C 12
BG008C 8
HU002C 6
HU003C 8
FI006C 2
BG009C 6
BG010C 8
FI003C 3
FR022C 7
FR023C 5
FR024C 9
FI007C 2
FI008C 2
DE074C 6
DE0

In [91]:
df_all

,urau_name,env_zone
0,CH011C,6
1,CH012C,6
2,BE003C,7
3,BE004C,7
4,BG018C,8
...,...,...
723,RO028C,8
724,SK001C,8
725,RO010C,6
726,SK002C,6


In [81]:
import shapely.geometry
bbox = bboxs[10]
IPython.display.GeoJSON(box(*bbox.transform_bounds(CRS.WGS84)).__geo_interface__)

<IPython.display.GeoJSON object>

In [93]:
import sqlite3
# save to sqlite
dbfile = r'./../../../s3/data/c001_city_cube/C_urban_cube_sh.sqlite'
conn = sqlite3.connect(dbfile) 
df_all.to_sql("c_city_env_zones",conn,if_exists='replace',index=False)
conn.commit()
conn.close()

In [94]:
#check database
query = """ 
  SELECT *
         FROM  c_city_env_zones
         LIMIT 10
         """
con = sqlite3.connect(dbfile)
df= pd.read_sql(query, con)
# Be sure to close the connection
con.close()
print (df)
print ("END")

  urau_name  env_zone
0    CH011C         6
1    CH012C         6
2    BE003C         7
3    BE004C         7
4    BG018C         8
5    FR027C         7
6    FR028C        13
7    FR029C         7
8    FR030C         0
9    FR044C         7
END
